# On Multiple Linear Regression - Codealong

In [1]:
# Let's begin by importing NumPy and Pandas



The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

## Dealing with Categorical Variables

One issue we'd like to resolve is what to do with categorical variables, i.e. variables that represent categories rather than continua. In a Pandas DataFrame, these columns may well have strings or objects for values, but they need not. Recall e.g. the heart-disease dataset from Kaggle in which the target variable took values 0-4, each representing a different stage of heart disease.

### Dummying

One very effective way of dealing with categorical variables is to dummy them out. What this involves is making a new column for _each categorical value in the column we're dummying out_. We'll do this below in our air safety dataset where we have a column of airline names.

These new columns will be filled only with 0's and 1's, a 1 representing the presence of the relevant categorical value.

Let's look at a simple example:

In [2]:
# Let's read in the chars dataset



In [4]:
# ... and look at it!



In [7]:
# Let's try using pd.get_dummies() to create our dummy columns:




# Now we need to add these dummy columns to our original dataset:



## Drug Use Dataset

In [8]:
# Let's read in the drug-use datset



In [9]:
# And check out the head



In [10]:
# Let's also look at drugs.info()



In [11]:
# Let's try mapping the age values to ints



What happened?

In [12]:
# Let's take a closer look at this 'age' column:



In [13]:
# Simple decision: We'll only use the first ten rows!



## Model Selection

Let's imagine that I'm going to try to predict age based on factors to do with drug use.

Now: Which columns (predictors) should I choose? Even ignoring the non-numeric categories in my dataset, there are still 20 predictors I could choose! For each of these predictors, I could either use it or not use it in my model, which means that there are 2^20 = 1,048,576 different models I could construct! Well, okay, one of these is the "empty model" with no predictors in it. But there are still 1,048,575 models from which I can choose!

How can I decide which predictors to use in my model?

### Correlation

In [14]:
# Use the .corr() DataFrame method to find out about the
# correlation values between all pairs of variables!




In [15]:
import seaborn as sns
sns.set(rc={'figure.figsize':(8, 8)})

# Use the .heatmap method to depict the relationships visually!




In [16]:
# Let's look at the correlations with 'age'
# (our dependent variable) in particular.



In [17]:
# Let's define X and y



### Multicollinearity

Probably 'alcohol-use' and 'alcohol-frequency' are highly correlated _with each other_ as well as with 'age'. This can lead to the production of an _overfit_ model. We'll stick a pin in this and return to the issue of overfit models soon.

## Multiple Regression in StatsModels

In [22]:
import statsmodels.api as sm

In [23]:
predictors = np.asarray(X)
predictors_int = sm.add_constant(predictors)
model = sm.OLS(y, predictors_int).fit()
model.summary()

/Users/gdamico/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     313.4
Date:                Thu, 11 Apr 2019   Prob (F-statistic):           5.56e-07
Time:                        09:36:20   Log-Likelihood:                0.56286
No. Observations:                  10   AIC:                             6.874
Df Residuals:                       6   BIC:                             8.085
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.9549      0.306     39.070      0.000      11.206      12.704
x1             0.0870      0.028      3.149      0.020       0.019       0.155
x2            -0.0052      0.008     -0.620      0.558      -0.026       0.015
x3             0.4206      0.502      0.837      0.435      -0.809       1.650
==============================================================================
Omnibus:                        0.433   Durbin-Watson:                   1.119
Prob(Omnibus):                  0.805   Jarque-Bera (JB):                0.472
Skew:                          -0.009   Prob(JB):                        0.790
Kurtosis:                       1.936   Cond. No.                         277.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Multiple Regression in Scikit-Learn

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [20]:
np.random.seed(33)

# Now let's split our data into train and test sets.



In [21]:
# Now we can fit the LinearRegression object to our training data!



In [22]:
# And score it on our testing set



In [23]:
# We can use the .coef_ attribute to recover the results
# of the regression.



## Recursive Feature Elimination

The idea behind recursive feature elimination is to build up (or down) to a small set of predictive features slowly, by eliminating the features with the lowest coefficients.

That is:
1. Start with a model with _all_ $n$ predictors;
2. find the predictor with the smallest coefficient;
3. throw that predictor out and build a model with the remining $n-1$ predictors;
4. set $n = n-1$ and repeat until $n-1$ has the value you want!

### Recursive Feature Elimination in Scikit-Learn

In [ ]:
from sklearn.feature_selection import RFE

lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=2)
select = select.fit(X = drugs.drop(columns=['age',
                                       'cocaine-frequency',
                                       'crack-frequency',
                                       'heroin-frequency',
                                       'inhalant-frequency',
                                       'oxycontin-frequency',
                                       'meth-frequency']),
                    y = drugs['age'])

select.ranking_

In [287]:
X2 = drugs[['meth-use', 'stimulant-use']]

### Sklearn Metrics

The metrics module in sklearn has a number of metrics that we can use to meaure the accuracy of our model, including the $R^2$ score, the mean absolute error and the mean squared error. Note that the default 'score' on our model object is the $R^2$ score.

In [170]:
metrics.r2_score(y_test, lr.predict(X_test))

0.9726930605104313

In [171]:
metrics.mean_absolute_error(y_test, lr.predict(X_test))

0.24770878321322107

In [172]:
metrics.mean_squared_error(y_test, lr.predict(X_test))

0.11529596673373455